# Elliptic Curve Math - Exercises with Libbitcoin BX

### 1. Basic EC operations

#### Derive a new valid secret  (secp256k1) from 512-bit entropy:

In [1]:
bx seed --bit_length 512 | bx ec-new


eab3637b383613be90049c388321a7f5a4be4e9adccd9a677b0ed5ece4b9b933


#### Derive a new (uncompressed) EC point from the generated secret:

In [2]:
bx ec-to-public --uncompressed f3e517af9d7665e8eb409be6c468d9b8e4d580ed1f5aeedcda9e05565c8502d0


041bc17dc0bd9e9e64a81be4bd9efa70941e30ada315583adcf67864142f6af47f7a17b3141508931fc55e26288cb7fb22a4ba34bed8109dbad3bbe978eef84f45


#### Demonstrate  associativity: 

Scalar operations: `(a + b) * c = a * c + b * c`
* `a = 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b`
* `b = b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2`
* `c = 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf`

In [3]:
# a + b
a_plus_b=$(bx ec-add-secrets 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2)

# left side: (a + b) * c
bx ec-multiply-secrets $a_plus_b 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf

# a * c 
a_mult_c=$(bx ec-multiply-secrets 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf)

# b * c
b_mult_c=$(bx ec-multiply-secrets b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2 6dbcfab245c6e278659dc26ec9d989c14c223f23cd17941ab45bb04c91290cdf)

# right side: a * c + b * c"
bx ec-add-secrets $a_mult_c $b_mult_c


5052a8db6fbe0325b2231b630b4a4ec42b9c8e0d7b00e62c5ed1569c85e2733f
5052a8db6fbe0325b2231b630b4a4ec42b9c8e0d7b00e62c5ed1569c85e2733f


Scalar & EC point operations: `(a + b) * G = A + B`

In [4]:
# a + b
a_plus_b=$(bx ec-add-secrets 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2)

# left side: (a + b) * G
bx ec-to-public $a_plus_b

# points A
A=$(bx ec-to-public 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b)

# right side: A + b * G
bx ec-add $A b6ab20f3d9311eb7ebcad6bb2933008eb350418c3f499fb764204ee08f4171f2


023cbf38233fd17a7c3126e14e85a03c27c81b97f3158511c0ae79011dad88a60b
023cbf38233fd17a7c3126e14e85a03c27c81b97f3158511c0ae79011dad88a60b


### 2. Joint public key point

`Alice` and `Bob` would like to create a `joint public key`. However, it is important that neither party alone has the corresponding private key of this joint public key point. How can this be achieved?

The joint public key generation scheme could follow the following sequence of events.

* `Alice` performs something.
* `Bob` performs something.
* `Alice` and `Bob` interact.
* `Alice` and `Bob` individually derive the `joint public key`

**`Alice` performs the following:**

In [2]:
# (a+b)G = A+B

alice_secret=$(bx seed --bit_length 512 | bx ec-new)
alice_point=$(bx ec-to-public $alice_secret)


 **`Bob` performs the following:**

In [3]:
bob_secret=$(bx seed --bit_length 512 | bx ec-new)
bob_point=$(bx ec-to-public $bob_secret)


#### `Alice` and `Bob` exchange the following data:

In [4]:
echo  "data from alice to bob: "
echo  $alice_point
echo  "data from bob to alice: "
echo  $bob_point


data from alice to bob: 
034dbb9ff6607d737cd7c43824f6f2943af5d4bd1ba5ee4664dc7aaf2e967bc650
data from bob to alice: 
02028dcc92320b942be026ead47451083bd07f646ea01a94f6ddf1609c5843acf9


#### `Alice` computes the following from data received from `Bob`

In [5]:
bx ec-add $bob_point $alice_secret


02d6123440efa4407a70b0dead58b7d29a3c1f1123c48640610d517f5f9e33e43d


#### `Bob` computes the following from data received from `Alice`

In [6]:
bx ec-add $alice_point $bob_secret


02d6123440efa4407a70b0dead58b7d29a3c1f1123c48640610d517f5f9e33e43d


### 2b. Spending from the joint public key 
How can `Alice` and `Bob` allow the other party to spend from this `joint public key`?

In [7]:
# Alice shares her secret with Bob.

# Bob computes the secret of the joint public key.
joint_secret=$(bx ec-add-secrets $alice_secret $bob_secret)

# Bob validates secret * G = joint_point
bx ec-to-public $joint_secret


02d6123440efa4407a70b0dead58b7d29a3c1f1123c48640610d517f5f9e33e43d


### 3. Commitment Schemes

`Alice` would like to commit to a secret number without revealing this number to `Bob`. This `commitment` must be binding, meaning `Alice` must be able to prove she hasn't changed it after the secret is revealed.

* `Alice` generates a commitment from a secret.
* `Bob` receives the commitment.
* `Alice` reveals the secret, proving it was the number originally commited to.

#### `Alice` generates a commitment from a secret number.

In [10]:
# With elliptic curve operations:
# -------------------------------
alice_secret=$(bx seed --bit_length 512 | bx ec-new)
alice_commitment=$(bx ec-to-public $alice_secret)
echo $alice_commitment

# With hashing operations:
# -------------------------------



0356e741407588d35d249eba186e1ebab32908c181870ef0a059cdb35332c91cf5


### 3b. How does `Bob` later verify Alice's commitment?

In [ ]:
# With elliptic curve operations:
# -------------------------------

# Step 1: Alice reveals secret to Bob.
# Step 2: Bob computes commitment from secret:
computed_commitment=$(bx ec-to-public $alice_secret)
echo $computed_commitment

# With hashing operations:
# -------------------------------



### 4) Blinding commitment Schemes

`Alice` creates multiple commitments for `Bob`, but would like prevent `Bob` from noticing if she uses the same secret multiple times. This is call a `blinding commitment` as it blinds the receiver of the commitment from gaining any information about the secret.

How can `Alice` achieve this?

* `Alice` generates a blinding commitment from a secret.
* `Bob` receives the commitment.
* `Alice` reveals the secret (and supporting data), proving it was the number originally commited to.

Alice's secret:
* `a = 86101c23edfcdf19bf47836b7fe4b86bd3023983d477e0324adc81230b22851b`

**Hint:** `Alice` can generate a random number to blind the commitment.

#### `Alice` generates a blinded commitment from a secret number.

In [1]:
# C = random_number * H + secret * G 
# C = random_mumber * e * G + secret * G = G * (random_number * e + secret)

# Alice generates a secret
secret=5a2a4e102e5440bb43887ab584591e0b975441d8ff940175a7a105e4dde85a4d

# Alice generates random_number 
random_number=$(bx seed --bit_length 512 | bx ec-new)

# Alice generates a hidden point H
hidden_point=$(bx seed --bit_length 512 | bx ec-new | bx ec-to-public)

# # random_number * H
r_mult_H=$(bx ec-multiply $hidden_point $random_number)

# C = random_number * H + secret * G
commitment=$(bx ec-add $r_mult_H $secret)
echo $commitment
echo $random_number


021c1e28b898a664e18ac52d8a6162a75d1d8b893036192eec460f2ab95932b7f0
8daea89dc67f76ea42465e1feba064132ae69b890e61cd194f2640b24a8081a2


#### `Bob` verifies that the secret matches her initial commitment.

In [4]:
# Alice reveals secret and random number to Bob
alice_secret=5a2a4e102e5440bb43887ab584591e0b975441d8ff940175a7a105e4dde85a4d
alice_random=8daea89dc67f76ea42465e1feba064132ae69b890e61cd194f2640b24a8081a2

# C = random_number * H + secret * G
r_mult_H=$(bx ec-multiply $hidden_point $alice_random)
computed_commitment=$(bx ec-add $r_mult_H $alice_secret)
echo $computed_commitment


021c1e28b898a664e18ac52d8a6162a75d1d8b893036192eec460f2ab95932b7f0
